- plan and implement a langchain project 
- examine and understand app.py
- note down further questions for Debasis
- name of app could be Pollard eSearch, 

1. I created a program that goes to the Pollard memorial library website, searches a user query in the searchbar and returns the titles in the first page
- need to understand the code thoroughly
- create plan to integrate langchain and llm processes
- ideas- user input frontend and search directly though link manipulation, more detailed results (like type of book, availability, etc), llm idea- provide summary of the book if it isn't there, toggle to other pages of results, do something with the preview pdf

2. Set up LLM in seperate cell- brainstorm
- set up LLM with openai and langchain
- the aim is to produce and display summaries for results lacking one by providing the details (tile, author, etc) to openai prompt
- seems quite simple and straightforward- there is low intersection between the webdriver function and summary creation process. Both are seperate and modularized

3. Langchain didn't work out for no because I do not have any free tokens left for openai
- next step is to finish a simple front end framework that user can use to retrieve search 

4. Time to retrieve and show is 4-5 seconds
- Error getting author info when searching Thomas Edison

In [69]:
# pip install selenium beautifulsoup4 requests

In [1]:
from selenium import webdriver # webdriver is responsible for controlling a web browser using Python code
from selenium.webdriver.common.by import By # By is used to locate elements in a page such as by ID, class name, tag name, etc
from selenium.webdriver.common.keys import Keys # provides keyboard key actions, such as pressing Enter
from selenium.webdriver.support.ui import WebDriverWait # imports WebDriverWait class which provides a way to wait until certain condition occurs to continue
from selenium.webdriver.support import expected_conditions as EC # used with webDriverWait to specify what you are waiting for
from selenium.webdriver.chrome.options import Options # to set options for Chrome WebDriver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate as CPT
from openai import RateLimitError
import time
import tkinter as tk

# global variable- for now
user_search = ""

class App():
    def __init__(self, view):
        self.start(view)
    def start(self, view):
        view.search()
        view.retrieve()
        view.show(view.books)
        
class View():
    # front-end processing and retrieval of search results
    def __init__(self, prompt):
        self.prompt = prompt
        self.books = []
        # call self.search() in App class
    def search(self):
        def on_button_click():
            global user_search
            user_search = search_box.get()  # Retrieve the text from the Entry widget
            loading_label.config(text="Loading...")
            window.update()
            window.destroy()
        window = tk.Tk()
        window.title("Pollard Memorial Search")
        label = tk.Label(window, text="Search by title or author:").pack(padx=20, pady=10)
        search_box = tk.Entry(window)
        search_box.pack(padx=20, pady=5)
        submit = tk.Button(window, text="Submit", command=on_button_click).pack(pady=5)
        loading_label = tk.Label(window, text="")
        loading_label.pack(pady=3)

        window.mainloop()
        return
    def retrieve(self):
        # Set up Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Enable headless mode
        chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (may not be necessary)
        chrome_options.add_argument("--window-size=1920x1080")  # Set window size for headless mode
        # Set up the WebDriver with headless options
        driver = webdriver.Chrome(options=chrome_options)
        driver.get("https://mvlc.ent.sirsi.net/client/en_US/lowell1/?dt=list")
        
        search_box = driver.find_element(By.NAME, 'q') 
        search_box.send_keys(user_search)
        search_box.send_keys(Keys.RETURN) 
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'results_cell'))
            )
        except Exception as e:
            print(f"No results found for '{user_search}': {e}")
            
        self.books = self.retrieve_helper(driver)
        driver.quit()
    def retrieve_helper(self, driver):
        books = []
        results = driver.find_elements(By.CLASS_NAME, 'results_cell')
        for result in results:
            try:
                title = result.find_element(By.CLASS_NAME, 'hideIE').text
                if result.find_element(By.CLASS_NAME, 'formatText').text == "eBook":
                    author = result.find_element(By.CLASS_NAME, 'INITIAL_AUTHOR_SRCH').text
                else:
                    author = result.find_element(By.CLASS_NAME, 'PERSONAL_AUTHOR').text
                summary = self.prompt.generate(title, author)
                book = Book(title, author, summary)
                books.append(book)
            except Exception as e:
                print(f"Error retrieving content: {e}")
        return books
    def show(self, books):
        window = tk.Tk()
        window.title(f"You searched: {user_search}")
        for i in range(len(books)):
            serial_num = i + 1
            label = tk.Label(window, text=f"{serial_num}. {books[i].to_string()}").pack(padx=20, pady=6)
        window.mainloop()
        
class Prompt():
    # manages interaction with openai to generate summaries for books
    def __init__(self):
        self.chat = self.chat_init()
        self.template = self.template_init()
    def chat_init(self):
        return ChatOpenAI(temperature=0.0, model = "gpt-3.5-turbo", openai_api_key="Enter-Here")
    def template_init(self):
        template_string = """Produce the blurb for the provided book details \
        that are delineated by three backsticks '''{title}, {author}'''
        """
        prompt_template = CPT.from_template(template_string)
        return prompt_template
    def query(self, book_title, book_author): 
        query = self.template.format_messages(
            title = book_title,
            author = book_author
        )
        return query
    def generate(self, title, author):
        # generates the response
        """
        try:
            query = self.query(title, author)
            response = self.chat.invoke(query)
        except RateLimitError as e:
            # print("Rate limit exceeded, try again later.")
            message = "Summary generation still in progress"
            return message 
        """
        return "None" # for now without API call

class Book():
    # Book class so that Book objects with various attributes can be created 
    def __init__(self, title, author, summary):
        self.title = title     # Instance variable for book title
        self.author = author   # Instance variable for book author
        self.summary = summary # Instance variable for book summary
    def to_string(self):
        return f"Title: {self.title}\nAuthor: {self.author}\nSummary: {self.summary}"

prompt = Prompt()
view = View(prompt)
Pollard = App(view)

Error retrieving content: Message: no such element: Unable to locate element: {"method":"css selector","selector":".PERSONAL_AUTHOR"}
  (Session info: chrome=128.0.6613.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001012cf9a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x00000001012c7e10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000100eccbe0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000100f10f10 cxxbridge1$string$len + 368852
4   chromedriver                        0x0000000100f073c0 cxxbridge1$string$len + 329092
5   chromedriver                        0x0000000100f4a4a4 cxxbridge1$string$len + 603752
6   chromedriver                        0x0000000100f05a08 cxxbridge1$string$len + 322508
7   chromedriver                        0x00

In [39]:
# trial code below- full unorganized library search, llm integration attempt, llm api key test, trying tkinter
from selenium import webdriver # webdriver is responsible for controlling a web browser using Python code
from selenium.webdriver.common.by import By # By is used to locate elements in a page such as by ID, class name, tag name, etc
from selenium.webdriver.common.keys import Keys # provides keyboard key actions, such as pressing Enter
from selenium.webdriver.support.ui import WebDriverWait # imports WebDriverWait class which provides a way to wait until certain condition occurs to continue
from selenium.webdriver.support import expected_conditions as EC # used with webDriverWait to specify what you are waiting for
import time
import tkinter as tk


# global variable
user_input = ""
def on_button_click():
    global user_input
    label = tk.Label(window, text="Loading...")
    label.pack(padx=20, pady=10)
    user_input = search_entry.get()  # Retrieve the text from the Entry widget
    result_label.config(text=f"You entered: {user_input}")  # Update the Label with the user input
    # window.destroy()

# Create the main window
window = tk.Tk()
window.title("Pollard Memorial Search")

# Create a label widget
label = tk.Label(window, text="Search by title or author:")
label.pack(padx=20, pady=10)

# Create entry widgets
search_entry = tk.Entry(window)
search_entry.pack(padx=20, pady=5)

# Create a button widget
button = tk.Button(window, text="Submit", command=on_button_click)
button.pack(pady=5)

# Create a Label widget to display the result
result_label = tk.Label(window, text="")
result_label.pack(pady=10)

global user_input
if user_input != "":
    window.destroy()

# Start the Tkinter event loop
window.mainloop()

# Set up the WebDriver- initializes the WebDriver by launching a new instance of the Chrome browser
driver = webdriver.Chrome() # (executable_path='/path/to/chromedriver')

# Open the website- tells the WebDriver to navigate to the specified URL
driver.get("https://mvlc.ent.sirsi.net/client/en_US/lowell1/?dt=list") # "https://mvlc.ent.sirsi.net/client/en_US/lowell1/search/results?qu=Python+programming&te="

# Locate and interact with the search box
search_box = driver.find_element(By.NAME, 'q') # locates the search box by its 'name' attributte and stores the element in search_box
# user_input placeholder here
search_box.send_keys(user_input) # this simulates typing the text into the box

# Submit the search
search_box.send_keys(Keys.RETURN) # like pressing enter- using send_keys

# Wait for results to load
# time.sleep(10)  # Wait for 10 seconds (or use WebDriverWait for better handling)
# uses WebDiverWait to wait up to 10 seconds for the search results to load- waits until an element with class name 'results_cell' is found on the page
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'results_cell'))
)

# Parse the results
results = driver.find_elements(By.CLASS_NAME, 'results_cell') # this finds all elements on the page with the class name 'results_cell'
titles = []
authors = []
for result in results:
    try:
        title = result.find_element(By.CLASS_NAME, 'hideIE').text
        titles.append(title)
    except Exception as e:
        print(f"Error retrieving title: {e}")
for result in results:
    try:
        if result.find_element(By.CLASS_NAME, 'formatText').text == "eBook":
            author = result.find_element(By.CLASS_NAME, 'INITIAL_AUTHOR_SRCH').text
        else:
            author = result.find_element(By.CLASS_NAME, 'PERSONAL_AUTHOR').text 
        authors.append(author)
    except Exception as e:
        print(f"Error retrieving author: {e}")

# Close the browser
driver.quit()

results_window = tk.Tk()
results_window.Title = f"You searched: {user_input}"
for title in titles:
    result_label = tk.Label(results_window, text=title)
    result_label.pack(padx=20, pady=5)
for author in authors:
    result_label = tk.Label(results_window, text=author)
    result_label.pack(padx=20, pady=5)

results_window.mainloop()

def config_search(self, search):
        # returns the user's search in the necessary format to access link using driver
        words = list(search.split(" "))
        string = ""
        for word in words:
            if word == words[-1]:
                string += word
                return string
            string += word + "+"

- Selenium supports automation of browsers through the use of a WebDriver
- WebDriver is an API and protocol that defines a language-neutral interface for controlling the behavior of web browsers
- Each browser is backed by a specific WebDriver implementation, called a driver (like chromeDriver)
- The driver is responsible for delegating down to the browser, and handles communication to and from Selenium and the browser
- 

In [47]:
# pip install --upgrade langchain
# pip install langchain_community
# pip install -U langchain-openai (-U tells pip to upgrade package to the latest version if necessary)
# pip install openai

In [67]:
# llm integration
# from langchain.chat_models import ChatOpenAI # ChatOpenAI depreciated
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate as CPT
from openai import RateLimitError

llm_model = "gpt-3.5-turbo"
OPENAI_API_KEY = "-Blank-"
USER_OPENAI_API_KEY = "-Blank-"
chat = ChatOpenAI(temperature=0.0, model = llm_model, openai_api_key=USER_OPENAI_API_KEY)

# creating prompt template using a template string- can add publication info
template_string = """Produce the blurb for the provided book details \
that are delineated by three backsticks '''{title}, {author}'''
"""
prompt_template = CPT.from_template(template_string)
# prompt_template.messages[0].prompt
# prompt_template.messages[0].prompt.input_variables

# formatted prompt to pass to openai
book_title = "Norwegian Wood"
book_author = "Haruki Murakami"
user_query = prompt_template.format_messages(
    title=book_title,
    author=book_author
)
# print(type(user_query))
# print(type(user_query[0]))
try:
    response = chat.invoke(user_query)
except RateLimitError as e:
    print("Rate limit exceeded, try again later.")

Rate limit exceeded, try again later.


In [59]:
import openai

openai.api_key = "-Blank-"

try:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Hello, world!"}]
    )
    print(response.choices[0].message['content'])
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [22]:
import tkinter as tk

# global variable
user_input = ""

def on_button_click():
    global user_input
    user_input = search_entry.get()  # Retrieve the text from the Entry widget
    result_label.config(text=f"You entered: {user_input}")  # Update the Label with the user input

# Create the main window
window = tk.Tk()
window.title("Pollard Memorial Search")

# Create a label widget
label = tk.Label(window, text="Search by title or author:")
label.pack(padx=20, pady=10)

# Create entry widgets
search_entry = tk.Entry(window)
search_entry.pack(padx=20, pady=5)

# Create a button widget
button = tk.Button(window, text="Submit", command=on_button_click)
button.pack(pady=5)

# Create a Label widget to display the result
result_label = tk.Label(window, text="")
result_label.pack(pady=10)

# Start the Tkinter event loop
window.mainloop()